<a href="https://colab.research.google.com/github/sumanjitraut/House_Price_Prediction/blob/main/House_Prices_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

house_prices_advanced_regression_techniques_path = kagglehub.competition_download('house-prices-advanced-regression-techniques')

print('Data source import complete.')


## 📂 Importing Libraries and Exploring Input Data

In this step, we:

- **Import core libraries**  
  - `numpy` → for linear algebra and numerical operations  
  - `pandas` → for data processing and CSV file I/O  

- **Explore the Kaggle input directory**  
  - Kaggle datasets are stored in the read‑only `../input/` folder.  
  - Using `os.walk`, we list all files available under this directory.  
  - This helps confirm the dataset files are accessible before loading them with `pandas`.

Running this cell will print the full paths of all files in the input directory, ensuring we know exactly what data is available for analysis.


In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_file_path = "/kaggle/input/house-prices-advanced-regression-techniques/train.csv"
dataset_df = pd.read_csv(train_file_path)
print("Full train dataset shape is {}".format(dataset_df.shape))

In [ ]:
list(set(dataset_df.dtypes.tolist()))

In [ ]:
df_num = dataset_df.select_dtypes(include = ['float64', 'int64'])
df_num.head()

In [ ]:
df_num.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8);

In [ ]:

#model = ydf.RandomForestLearner(label="l").train(ds)

## 📊 Model Evaluation with Train/Validation Split

In this step, we evaluate our RandomForest model using a hold‑out validation set:

1. **Split the dataset**  
   - We separate the training data into `X_train`/`y_train` and `X_valid`/`y_valid`.  
   - This allows us to measure how well the model generalizes to unseen data.

2. **Train the model**  
   - We fit a `RandomForestLearner` on the training split.  
   - The label is `SalePrice` and the task is regression.

3. **Predict on validation set**  
   - We generate predictions for `X_valid`.  
   - `.ravel()` flattens predictions into a 1D array.

4. **Compute RMSE (Root Mean Squared Error)**  
   - RMSE measures the average difference between predicted and actual sale prices.  
   - Lower RMSE indicates better performance.  
   - The result is printed as `Validation RMSE`.

This gives us a reliable offline metric before submitting

In [ ]:
import ydf
import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np

# Load train and test
train_ds_pd = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test_ds_pd  = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

model = ydf.RandomForestLearner(
    label="SalePrice",
    task=ydf.Task.REGRESSION   # use the enum, not a string
).train(train_ds_pd)

predictions = model.predict(test_ds_pd)

submission = pd.DataFrame({
    "Id": test_ds_pd["Id"],        # from test.csv
    "SalePrice": predictions       # now 1D
})

print(type(predictions))
print(predictions.shape)

submission.to_csv("/kaggle/working/submission.csv", index=False)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Split train into train/valid
train_features = train_ds_pd.drop(columns=["SalePrice"])
train_labels   = train_ds_pd["SalePrice"]

X_train, X_valid, y_train, y_valid = train_test_split(train_features, train_labels, test_size=0.2, random_state=42)

# Train model on training split
model = ydf.RandomForestLearner(
    label="SalePrice",
    task=ydf.Task.REGRESSION
).train(pd.concat([X_train, y_train], axis=1))

# Predict on validation split
valid_predictions = model.predict(X_valid).ravel()

# Compute RMSE
rmse_valid = np.sqrt(mean_squared_error(y_valid, valid_predictions))
print("Validation RMSE:", rmse_valid)
